# Deep Reinforcement Learning for Stock Trading from Scratch: Portfolio Allocation

Tutorials to use OpenAI DRL to perform portfolio allocation in one Jupyter Notebook | Presented at NeurIPS 2020: Deep RL Workshop

* This blog is based on our paper: FinRL: A Deep Reinforcement Learning Library for Automated Stock Trading in Quantitative Finance, presented at NeurIPS 2020: Deep RL Workshop.
* Check out medium blog for detailed explanations: https://towardsdatascience.com/finrl-for-quantitative-finance-tutorial-for-portfolio-allocation-9b417660c7cd
* Please report any issues to our Github: https://github.com/AI4Finance-Foundation/FinRL/issues
* **Pytorch Version**



# Content

* [1. Problem Definition](#0)
* [2. Getting Started - Load Python packages](#1)
    * [2.1. Install Packages](#1.1)    
    * [2.2. Check Additional Packages](#1.2)
    * [2.3. Import Packages](#1.3)
    * [2.4. Create Folders](#1.4)
* [3. Download Data](#2)
* [4. Preprocess Data](#3)        
    * [4.1. Technical Indicators](#3.1)
    * [4.2. Perform Feature Engineering](#3.2)
* [5.Build Environment](#4)  
    * [5.1. Training & Trade Data Split](#4.1)
    * [5.2. User-defined Environment](#4.2)   
    * [5.3. Initialize Environment](#4.3)    
* [6.Implement DRL Algorithms](#5)  
* [7.Backtesting Performance](#6)  
    * [7.1. BackTestStats](#6.1)
    * [7.2. BackTestPlot](#6.2)   
    * [7.3. Baseline Stats](#6.3)   
    * [7.3. Compare to Stock Market Index](#6.4)             

<a id='0'></a>
# Part 1. Problem Definition

This problem is to design an automated trading solution for portfolio alloacation. We model the stock trading process as a Markov Decision Process (MDP). We then formulate our trading goal as a maximization problem.

The algorithm is trained using Deep Reinforcement Learning (DRL) algorithms and the components of the reinforcement learning environment are:


* Action: The action space describes the allowed actions that the agent interacts with the
environment. Normally, a ∈ A represents the weight of a stock in the porfolio: a ∈ (-1,1). Assume our stock pool includes N stocks, we can use a list [a<sub>1</sub>, a<sub>2</sub>, ... , a<sub>N</sub>] to determine the weight for each stock in the porfotlio, where a<sub>i</sub> ∈ (-1,1), a<sub>1</sub>+ a<sub>2</sub>+...+a<sub>N</sub>=1. For example, "The weight of AAPL in the portfolio is 10%." is [0.1 , ...].

* Reward function: r(s, a, s′) is the incentive mechanism for an agent to learn a better action. The change of the portfolio value when action a is taken at state s and arriving at new state s',  i.e., r(s, a, s′) = v′ − v, where v′ and v represent the portfolio
values at state s′ and s, respectively

* State: The state space describes the observations that the agent receives from the environment. Just as a human trader needs to analyze various information before executing a trade, so
our trading agent observes many different features to better learn in an interactive environment.

* Environment: Dow 30 consituents


The data of the single stock that we will be using for this case study is obtained from Yahoo Finance API. The data contains Open-High-Low-Close price and volume.


<a id='1'></a>
# Part 2. Getting Started- Load Python Packages

<a id='1.1'></a>
## 2.1. Install all the packages through FinRL library


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
## install finrl library
!pip install wrds
!pip install swig
!pip install -q condacolab
import condacolab
condacolab.install()
!apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 14.2 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
  Attempting uninstall: packaging
    Found existing installation: packaging 24.1
    Uninstalling packaging-24.1:
      Successfully uninstalled packaging-24.1
  Attempting uninstall: pandas
    Found existing installation: pandas 2.1.4
    Uninstalling pandas-2.1.4:
      Successfully uninstalled pandas-2.1.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the 


<a id='1.2'></a>
## 2.2. Check if the additional packages needed are present, if not install them.
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Import Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
%matplotlib inline
import datetime

from finrl import config
from finrl import config_tickers
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_portfolio_allocation.env_portfolio import StockPortfolioEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline,convert_daily_return_to_pyfolio_ts
from finrl.meta.data_processor import DataProcessor
from finrl.meta.data_processors.processor_yahoofinance import YahooFinanceProcessor
import sys
sys.path.append("../FinRL-Library")

/usr/local/lib/python3.10/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


<a id='1.4'></a>
## 2.4. Create Folders

In [2]:
import os

print(config.DATA_SAVE_DIR)
print(config.TRAINED_MODEL_DIR)
print(config.TENSORBOARD_LOG_DIR)
print(config.RESULTS_DIR)

os.makedirs('/content/drive/MyDrive/'+config.DATA_SAVE_DIR, exist_ok=True)
os.makedirs('/content/drive/MyDrive/'+config.TRAINED_MODEL_DIR, exist_ok=True)
os.makedirs('/content/drive/MyDrive/'+config.TENSORBOARD_LOG_DIR, exist_ok=True)
os.makedirs('/content/drive/MyDrive/'+config.RESULTS_DIR, exist_ok=True)


datasets
trained_models
tensorboard_log
results


In [ ]:
import os
if not os.path.exists("/content/drive/Mydrive/" + config.DATA_SAVE_DIR):
    os.makedirs("/content/drive/Mydrive/" + config.DATA_SAVE_DIR)
if not os.path.exists("/content/drive/Mydrive/" + config.TRAINED_MODEL_DIR):
    os.makedirs("/content/drive/Mydrive/" + config.TRAINED_MODEL_DIR)
if not os.path.exists("/content/drive/Mydrive/" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("/content/drive/Mydrive/" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("/content/drive/Mydrive/" + config.RESULTS_DIR):
    os.makedirs("/content/drive/Mydrive/" + config.RESULTS_DIR)

<a id='2'></a>
# Part 3. Download Data
Yahoo Finance is a website that provides stock data, financial news, financial reports, etc. All the data provided by Yahoo Finance is free.
* FinRL uses a class **YahooDownloader** to fetch data from Yahoo Finance API
* Call Limit: Using the Public API (without authentication), you are limited to 2,000 requests per hour per IP (or up to a total of 48,000 requests a day).


In [3]:
print(config_tickers.DOW_30_TICKER)

['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']


In [4]:
import yfinance as yf
all_stock_tickers = yf.download(config_tickers.DOW_30_TICKER, start='2008-01-01', end='2023-12-31')
all_stock_tickers.head()

[*********************100%***********************]  30 of 30 completed


Price                     Adj Close                                   \
Ticker                         AAPL       AMGN        AXP         BA   
Date                                                                   
2008-01-02 00:00:00+00:00  5.876340  33.033276  39.221809  63.481617   
2008-01-03 00:00:00+00:00  5.879055  32.388199  38.737694  63.745438   
2008-01-04 00:00:00+00:00  5.430278  31.757309  37.761753  62.895348   
2008-01-07 00:00:00+00:00  5.357591  32.175541  37.930813  60.733353   
2008-01-08 00:00:00+00:00  5.164873  32.841877  36.847298  58.564014   

Price                                                                  \
Ticker                           CAT        CRM       CSCO        CVX   
Date                                                                    
2008-01-02 00:00:00+00:00  45.046818  15.026949  17.845200  48.668549   
2008-01-03 00:00:00+00:00  44.887390  15.201449  17.986399  49.267406   
2008-01-04 00:00:00+00:00  43.707489  14.672963  17.562798  48.611267   
2008-01-07 00:00:00+00:00  43.835049  14.398748  17.569519  47.975945   
2008-01-08 00:00:00+00:00  42.827316  13.635934  17.098848  47.361469   

Price                                     ...    Volume                      \
Ticker                           DIS DOW  ...       MRK      MSFT       NKE   
Date                                      ...                                 
2008-01-02 00:00:00+00:00  26.801105 NaN  ...  12287276  63004200   9781600   
2008-01-03 00:00:00+00:00  26.742186 NaN  ...   8831810  49599600  10636000   
2008-01-04 00:00:00+00:00  26.203472 NaN  ...  10469625  72090800  22335200   
2008-01-07 00:00:00+00:00  26.228722 NaN  ...  11874783  80164300  20808400   
2008-01-08 00:00:00+00:00  25.706839 NaN  ...  21523824  79148300  16714400   

Price                                                                          \
Ticker                           PG      TRV      UNH   V        VZ       WBA   
Date                                                                            
2008-01-02 00:00:00+00:00  12255300  2341400  7698700 NaN  10865660   7298800   
2008-01-03 00:00:00+00:00  11162800  2098800  4864000 NaN   9017975  16204900   
2008-01-04 00:00:00+00:00  10049000  3048900  7250600 NaN  11886820  11552400   
2008-01-07 00:00:00+00:00  11916000  3130300  6093200 NaN  16094216   7723200   
2008-01-08 00:00:00+00:00  17169100  4684600  8142700 NaN  37362464   9053600   

Price                                
Ticker                          WMT  
Date                                 
2008-01-02 00:00:00+00:00  59755800  
2008-01-03 00:00:00+00:00  59403000  
2008-01-04 00:00:00+00:00  61053600  
2008-01-07 00:00:00+00:00  60979500  
2008-01-08 00:00:00+00:00  57051300  

[5 rows x 180 columns]

In [5]:
# all_stock_tickers.reset_index(inplace=True)
new_df = all_stock_tickers.T
new_df.reset_index(inplace=True)
#new_df.set_index('Ticker', inplace=True)


new_df = new_df.melt(id_vars=['Price','Ticker'])
new_df = new_df.set_index(['Date','Ticker','Price'])
new_df = new_df.sort_index(level=['Date','Ticker','Price'], ascending=[True,True,True])
new_df = new_df.unstack(level=2)
new_df = new_df.droplevel(0, axis=1)
new_df = new_df.reset_index()
new_df.columns.name = None
new_df.columns = ['date','tic','adjcp','close','high','low','open','volume']
new_df['date'] = new_df['date'].dt.strftime('%Y-%m-%d')
new_df.head()
new_df

,date,tic,adjcp,close,high,low,open,volume
0,2008-01-02,AAPL,5.876340,6.958571,7.152143,6.876786,7.116786,1.079179e+09
1,2008-01-02,AMGN,33.033276,46.599998,47.040001,46.259998,46.599998,7.934400e+06
2,2008-01-02,AXP,39.221809,51.040001,52.320000,50.790001,52.090000,8.053700e+06
3,2008-01-02,BA,63.481617,86.620003,87.839996,86.000000,87.570000,4.303000e+06
4,2008-01-02,CAT,45.046818,70.629997,72.669998,70.050003,72.559998,6.337800e+06
...,...,...,...,...,...,...,...,...
120805,2023-12-29,UNH,522.187439,526.469971,528.239990,523.919983,525.979980,2.080100e+06
120806,2023-12-29,V,258.866302,260.350006,261.459991,259.670013,260.570007,4.074600e+06
120807,2023-12-29,VZ,35.896526,37.700001,37.759998,37.320000,37.380001,1.620520e+07
120808,2023-12-29,WBA,24.868887,26.110001,26.629999,25.750000,26.440001,1.085330e+07


In [ ]:
dp = YahooFinanceProcessor()
df = dp.download_data(start_date = '2008-01-01',
                     end_date = '2023-12-31',
                     ticker_list = config_tickers.DOW_30_TICKER, time_interval='1D')

In [6]:
df = new_df
df.head()

,date,tic,adjcp,close,high,low,open,volume
0,2008-01-02,AAPL,5.876340,6.958571,7.152143,6.876786,7.116786,1.079179e+09
1,2008-01-02,AMGN,33.033276,46.599998,47.040001,46.259998,46.599998,7.934400e+06
2,2008-01-02,AXP,39.221809,51.040001,52.320000,50.790001,52.090000,8.053700e+06
3,2008-01-02,BA,63.481617,86.620003,87.839996,86.000000,87.570000,4.303000e+06
4,2008-01-02,CAT,45.046818,70.629997,72.669998,70.050003,72.559998,6.337800e+06


In [7]:
df.shape

(120810, 8)

# Part 4: Preprocess Data
Data preprocessing is a crucial step for training a high quality machine learning model. We need to check for missing data and do feature engineering in order to convert the data into a model-ready state.
* Add technical indicators. In practical trading, various information needs to be taken into account, for example the historical stock prices, current holding shares, technical indicators, etc. In this article, we demonstrate two trend-following technical indicators: MACD and RSI.
* Add turbulence index. Risk-aversion reflects whether an investor will choose to preserve the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the financial turbulence index that measures extreme asset price fluctuation.

In [8]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    use_turbulence=False,
                    user_defined_feature = False)

df = fe.preprocess_data(df)

Successfully added technical indicators


In [9]:
df.shape

(112756, 16)

In [10]:
df.head()

,date,tic,adjcp,close,high,low,open,volume,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
0,2008-01-02,AAPL,5.876340,6.958571,7.152143,6.876786,7.116786,1.079179e+09,0.0,6.964725,6.955632,100.0,-66.666667,100.0,6.958571,6.958571
4027,2008-01-02,AMGN,33.033276,46.599998,47.040001,46.259998,46.599998,7.934400e+06,0.0,6.964725,6.955632,100.0,-66.666667,100.0,46.599998,46.599998
8054,2008-01-02,AXP,39.221809,51.040001,52.320000,50.790001,52.090000,8.053700e+06,0.0,6.964725,6.955632,100.0,-66.666667,100.0,51.040001,51.040001
12081,2008-01-02,BA,63.481617,86.620003,87.839996,86.000000,87.570000,4.303000e+06,0.0,6.964725,6.955632,100.0,-66.666667,100.0,86.620003,86.620003
16108,2008-01-02,CAT,45.046818,70.629997,72.669998,70.050003,72.559998,6.337800e+06,0.0,6.964725,6.955632,100.0,-66.666667,100.0,70.629997,70.629997


## Add covariance matrix as states

In [11]:
# add covariance matrix as states
df=df.sort_values(['date','tic'],ignore_index=True)
df.index = df.date.factorize()[0]

cov_list = []
return_list = []

# look back is one year
lookback=252
for i in range(lookback,len(df.index.unique())):
  data_lookback = df.loc[i-lookback:i,:]
  price_lookback=data_lookback.pivot_table(index = 'date',columns = 'tic', values = 'close')
  return_lookback = price_lookback.pct_change().dropna()
  return_list.append(return_lookback)

  covs = return_lookback.cov().values
  cov_list.append(covs)


df_cov = pd.DataFrame({'date':df.date.unique()[lookback:],'cov_list':cov_list,'return_list':return_list})
df = df.merge(df_cov, on='date')
df = df.sort_values(['date','tic']).reset_index(drop=True)


In [12]:
df.shape

(105700, 18)

In [13]:
df.head()

,date,tic,adjcp,close,high,low,open,volume,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,cov_list,return_list
0,2008-12-31,AAPL,2.574141,3.048214,3.133571,3.047857,3.070357,607541200.0,-0.097446,3.649552,2.895305,42.254771,-80.847207,16.129793,3.243631,3.375887,"[[0.001348968986171653, 0.00042841264280825875...",tic AAPL AMGN AXP ...
1,2008-12-31,AMGN,40.937157,57.750000,58.220001,57.060001,57.110001,6287200.0,0.216368,58.947401,56.388599,51.060614,51.895357,10.432018,56.671334,56.044333,"[[0.001348968986171653, 0.00042841264280825875...",tic AAPL AMGN AXP ...
2,2008-12-31,AXP,14.445875,18.549999,18.750000,17.910000,17.969999,9625600.0,-1.191668,23.723023,16.106977,42.521170,-74.811722,25.776759,20.030000,22.412000,"[[0.001348968986171653, 0.00042841264280825875...",tic AAPL AMGN AXP ...
3,2008-12-31,BA,32.005890,42.669998,43.049999,41.500000,41.590000,5443100.0,-0.391219,42.894634,38.486366,47.290375,157.922391,5.366299,40.432000,43.304500,"[[0.001348968986171653, 0.00042841264280825875...",tic AAPL AMGN AXP ...
4,2008-12-31,CAT,29.245958,44.669998,45.099998,43.700001,43.700001,6277400.0,0.979845,45.785565,38.404435,51.073052,98.904653,26.331746,40.266000,39.918333,"[[0.001348968986171653, 0.00042841264280825875...",tic AAPL AMGN AXP ...


<a id='4'></a>
# Part 5. Design Environment
Considering the stochastic and interactive nature of the automated stock trading tasks, a financial task is modeled as a **Markov Decision Process (MDP)** problem. The training process involves observing stock price change, taking an action and reward's calculation to have the agent adjusting its strategy accordingly. By interacting with the environment, the trading agent will derive a trading strategy with the maximized rewards as time proceeds.

Our trading environments, based on OpenAI Gym framework, simulate live stock markets with real market data according to the principle of time-driven simulation.


## Training data split: 2009-01-01 to 2020-07-01

In [14]:
train = data_split(df, '2009-01-01','2020-07-01')
#trade = data_split(df, '2020-01-01', config.END_DATE)

In [15]:
train.head()

,date,tic,adjcp,close,high,low,open,volume,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,cov_list,return_list
0,2009-01-02,AAPL,2.737004,3.241071,3.251429,3.041429,3.067143,746015200.0,-0.082758,3.633600,2.892864,45.440193,-30.508777,2.140064,3.244631,3.376833,"[[0.001366150662406761, 0.000433938195725591, ...",tic AAPL AMGN AXP ...
0,2009-01-02,AMGN,41.816158,58.990002,59.080002,57.750000,58.590000,6547900.0,0.320448,59.148360,56.339640,52.756859,94.549630,0.814217,56.759667,56.166000,"[[0.001366150662406761, 0.000433938195725591, ...",tic AAPL AMGN AXP ...
0,2009-01-02,AXP,15.053308,19.330000,19.520000,18.400000,18.570000,10955700.0,-1.059847,23.489423,16.086577,43.923322,-42.018825,16.335101,20.028333,22.263333,"[[0.001366150662406761, 0.000433938195725591, ...",tic AAPL AMGN AXP ...
0,2009-01-02,BA,33.941093,45.250000,45.560001,42.779999,42.799999,7010200.0,-0.019566,43.926849,37.932151,50.664690,275.696308,20.494464,40.621667,43.237334,"[[0.001366150662406761, 0.000433938195725591, ...",tic AAPL AMGN AXP ...
0,2009-01-02,CAT,30.712521,46.910000,46.980000,44.709999,44.910000,7117200.0,1.248426,46.543072,38.372928,53.534743,131.675975,34.637448,40.623333,39.911333,"[[0.001366150662406761, 0.000433938195725591, ...",tic AAPL AMGN AXP ...


## Environment for Portfolio Allocation


In [16]:
import numpy as np
import pandas as pd
from gym.utils import seeding
import gym
from gym import spaces
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from stable_baselines3.common.vec_env import DummyVecEnv


class StockPortfolioEnv(gym.Env):
    """A single stock trading environment for OpenAI gym

    Attributes
    ----------
        df: DataFrame
            input data
        stock_dim : int
            number of unique stocks
        hmax : int
            maximum number of shares to trade
        initial_amount : int
            start money
        transaction_cost_pct: float
            transaction cost percentage per trade
        reward_scaling: float
            scaling factor for reward, good for training
        state_space: int
            the dimension of input features
        action_space: int
            equals stock dimension
        tech_indicator_list: list
            a list of technical indicator names
        turbulence_threshold: int
            a threshold to control risk aversion
        day: int
            an increment number to control date

    Methods
    -------
    _sell_stock()
        perform sell action based on the sign of the action
    _buy_stock()
        perform buy action based on the sign of the action
    step()
        at each step the agent will return actions, then
        we will calculate the reward, and return the next observation.
    reset()
        reset the environment
    render()
        use render to return other functions
    save_asset_memory()
        return account value at each time step
    save_action_memory()
        return actions/positions at each time step


    """
    metadata = {'render.modes': ['human']}

    def __init__(self,
                df,
                stock_dim,
                hmax,
                initial_amount,
                transaction_cost_pct,
                reward_scaling,
                state_space,
                action_space,
                tech_indicator_list,
                turbulence_threshold=None,
                lookback=252,
                day = 0):
        #super(StockEnv, self).__init__()
        #money = 10 , scope = 1
        self.day = day
        self.lookback=lookback
        self.df = df
        self.stock_dim = stock_dim
        self.hmax = hmax
        self.initial_amount = initial_amount
        self.transaction_cost_pct =transaction_cost_pct
        self.reward_scaling = reward_scaling
        self.state_space = state_space
        self.action_space = action_space
        self.tech_indicator_list = tech_indicator_list

        # action_space normalization and shape is self.stock_dim
        self.action_space = spaces.Box(low = 0, high = 1,shape = (self.action_space,))
        # Shape = (34, 30)
        # covariance matrix + technical indicators
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape = (self.state_space+len(self.tech_indicator_list),self.state_space))

        # load data from a pandas dataframe
        self.data = self.df.loc[self.day,:]
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.terminal = False
        self.turbulence_threshold = turbulence_threshold
        # initalize state: inital portfolio return + individual stock return + individual weights
        self.portfolio_value = self.initial_amount

        # memorize portfolio value each step
        self.asset_memory = [self.initial_amount]
        # memorize portfolio return each step
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]


    def step(self, actions):
        # print(self.day)
        self.terminal = self.day >= len(self.df.index.unique())-1
        # print(actions)

        if self.terminal:
            df = pd.DataFrame(self.portfolio_return_memory)
            df.columns = ['daily_return']
            plt.plot(df.daily_return.cumsum(),'r')
            plt.savefig('drive/MyDrive/results/cumulative_reward.png')
            plt.close()

            plt.plot(self.portfolio_return_memory,'r')
            plt.savefig('drive/MyDrive/results/rewards.png')
            plt.close()

            print("=================================")
            print("begin_total_asset:{}".format(self.asset_memory[0]))
            print("end_total_asset:{}".format(self.portfolio_value))

            df_daily_return = pd.DataFrame(self.portfolio_return_memory)
            df_daily_return.columns = ['daily_return']
            if df_daily_return['daily_return'].std() !=0:
              sharpe = (252**0.5)*df_daily_return['daily_return'].mean()/ \
                       df_daily_return['daily_return'].std()
              print("Sharpe: ",sharpe)
            print("=================================")

            return self.state, self.reward, self.terminal,{}

        else:
            #print("Model actions: ",actions)
            # actions are the portfolio weight
            # normalize to sum of 1
            #if (np.array(actions) - np.array(actions).min()).sum() != 0:
            #  norm_actions = (np.array(actions) - np.array(actions).min()) / (np.array(actions) - np.array(actions).min()).sum()
            #else:
            #  norm_actions = actions
            weights = self.softmax_normalization(actions)
            #print("Normalized actions: ", weights)
            self.actions_memory.append(weights)
            last_day_memory = self.data

            #load next state
            self.day += 1
            self.data = self.df.loc[self.day,:]
            self.covs = self.data['cov_list'].values[0]
            self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
            #print(self.state)
            # calcualte portfolio return
            # individual stocks' return * weight
            portfolio_return = sum(((self.data.close.values / last_day_memory.close.values)-1)*weights)
            # update portfolio value
            new_portfolio_value = self.portfolio_value*(1+portfolio_return)
            self.portfolio_value = new_portfolio_value

            # save into memory
            self.portfolio_return_memory.append(portfolio_return)
            self.date_memory.append(self.data.date.unique()[0])
            self.asset_memory.append(new_portfolio_value)

            # the reward is the new portfolio value or end portfolo value
            self.reward = new_portfolio_value
            #print("Step reward: ", self.reward)
            #self.reward = self.reward*self.reward_scaling

        return self.state, self.reward, self.terminal, {}

    def reset(self):
        self.asset_memory = [self.initial_amount]
        self.day = 0
        self.data = self.df.loc[self.day,:]
        # load states
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.portfolio_value = self.initial_amount
        #self.cost = 0
        #self.trades = 0
        self.terminal = False
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]
        return self.state

    def render(self, mode='human'):
        return self.state

    def softmax_normalization(self, actions):
        numerator = np.exp(actions)
        denominator = np.sum(np.exp(actions))
        softmax_output = numerator/denominator
        return softmax_output


    def save_asset_memory(self):
        date_list = self.date_memory
        portfolio_return = self.portfolio_return_memory
        #print(len(date_list))
        #print(len(asset_list))
        df_account_value = pd.DataFrame({'date':date_list,'daily_return':portfolio_return})
        return df_account_value

    def save_action_memory(self):
        # date and close price length must match actions length
        date_list = self.date_memory
        df_date = pd.DataFrame(date_list)
        df_date.columns = ['date']

        action_list = self.actions_memory
        df_actions = pd.DataFrame(action_list)
        df_actions.columns = self.data.tic.values
        df_actions.index = df_date.date
        #df_actions = pd.DataFrame({'date':date_list,'actions':action_list})
        return df_actions

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def get_sb_env(self):
        e = DummyVecEnv([lambda: self])
        obs = e.reset()
        return e, obs

In [17]:
stock_dimension = len(train.tic.unique())
state_space = stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")


Stock Dimension: 28, State Space: 28


In [18]:
env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "transaction_cost_pct": 0.001,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": config.INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4

}

e_train_gym = StockPortfolioEnv(df = train, **env_kwargs)

In [19]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


/usr/local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


<a id='5'></a>
# Part 6: Implement DRL Algorithms
* The implementation of the DRL algorithms are based on **OpenAI Baselines** and **Stable Baselines**. Stable Baselines is a fork of OpenAI Baselines, with a major structural refactoring, and code cleanups.
* FinRL library includes fine-tuned standard DRL algorithms, such as DQN, DDPG,
Multi-Agent DDPG, PPO, SAC, A2C and TD3. We also allow users to
design their own DRL algorithms by adapting these DRL algorithms.

In [20]:
# initialize
agent = DRLAgent(env = env_train)

### Model 1: **A2C**


In [21]:
agent = DRLAgent(env = env_train)

A2C_PARAMS = {"n_steps": 5, "ent_coef": 0.005, "learning_rate": 0.0002}
model_a2c = agent.get_model(model_name="a2c",model_kwargs = A2C_PARAMS)

{'n_steps': 5, 'ent_coef': 0.005, 'learning_rate': 0.0002}
Using cuda device


In [22]:
trained_a2c = agent.train_model(model=model_a2c,
                                tb_log_name='a2c',
                                total_timesteps=50000)

-------------------------------------
| time/                 |           |
|    fps                | 230       |
|    iterations         | 100       |
|    time_elapsed       | 2         |
|    total_timesteps    | 500       |
| train/                |           |
|    entropy_loss       | -39.7     |
|    explained_variance | 1.19e-07  |
|    learning_rate      | 0.0002    |
|    n_updates          | 99        |
|    policy_loss        | 1.96e+08  |
|    reward             | 1652822.8 |
|    std                | 0.999     |
|    value_loss         | 2.88e+13  |
-------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 260       |
|    iterations         | 200       |
|    time_elapsed       | 3         |
|    total_timesteps    | 1000      |
| train/                |           |
|    entropy_loss       | -39.7     |
|    explained_variance | 0         |
|    learning_rate      | 0.0002    |
|    n_updat

In [23]:
trained_a2c.save('/content/drive/MyDrive/trained_models/trained_a2c.zip')

### Model 2: **PPO**


In [24]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.005,
    "learning_rate": 0.0001,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

{'n_steps': 2048, 'ent_coef': 0.005, 'learning_rate': 0.0001, 'batch_size': 128}
Using cuda device


In [25]:
trained_ppo = agent.train_model(model=model_ppo,
                             tb_log_name='ppo',
                             total_timesteps=80000)

----------------------------------
| time/              |           |
|    fps             | 397       |
|    iterations      | 1         |
|    time_elapsed    | 5         |
|    total_timesteps | 2048      |
| train/             |           |
|    reward          | 3630745.0 |
----------------------------------
begin_total_asset:1000000
end_total_asset:4850214.318624085
Sharpe:  0.8438787629914672
-------------------------------------------
| time/                   |               |
|    fps                  | 327           |
|    iterations           | 2             |
|    time_elapsed         | 12            |
|    total_timesteps      | 4096          |
| train/                  |               |
|    approx_kl            | 1.2019882e-08 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -39.7         |
|    explained_variance   | 0             |
|    learning_rate        | 0.0001        |
|    loss                

In [26]:
trained_ppo.save('/content/trained_models/trained_ppo.zip')

/usr/local/lib/python3.10/site-packages/stable_baselines3/common/save_util.py:284: UserWarning: Path '/content/trained_models' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


### Model 3: **DDPG**


In [27]:
agent = DRLAgent(env = env_train)
DDPG_PARAMS = {"batch_size": 128, "buffer_size": 50000, "learning_rate": 0.001}


model_ddpg = agent.get_model("ddpg",model_kwargs = DDPG_PARAMS)

{'batch_size': 128, 'buffer_size': 50000, 'learning_rate': 0.001}
Using cuda device


In [28]:
trained_ddpg = agent.train_model(model=model_ddpg,
                             tb_log_name='ddpg',
                             total_timesteps=50000)

begin_total_asset:1000000
end_total_asset:4795642.988817178
Sharpe:  0.8348780223095964
begin_total_asset:1000000
end_total_asset:4754400.849767472
Sharpe:  0.8264818507392208
begin_total_asset:1000000
end_total_asset:4754400.849767472
Sharpe:  0.8264818507392208
begin_total_asset:1000000
end_total_asset:4754400.849767472
Sharpe:  0.8264818507392208
----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 109       |
|    time_elapsed    | 106       |
|    total_timesteps | 11572     |
| train/             |           |
|    actor_loss      | -1.13e+08 |
|    critic_loss     | 1.62e+12  |
|    learning_rate   | 0.001     |
|    n_updates       | 11471     |
|    reward          | 4754401.0 |
----------------------------------
begin_total_asset:1000000
end_total_asset:4754400.849767472
Sharpe:  0.8264818507392208
begin_total_asset:1000000
end_total_asset:4754400.849767472
Sharpe:  0.8264818507392208
begin_total_asset

In [29]:
trained_ddpg.save('/content/trained_models/trained_ddpg.zip')

### Model 4: **SAC**


In [30]:
agent = DRLAgent(env = env_train)
SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 100000,
    "learning_rate": 0.0003,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

{'batch_size': 128, 'buffer_size': 100000, 'learning_rate': 0.0003, 'learning_starts': 100, 'ent_coef': 'auto_0.1'}
Using cuda device


In [31]:
trained_sac = agent.train_model(model=model_sac,
                             tb_log_name='sac',
                             total_timesteps=50000)

begin_total_asset:1000000
end_total_asset:4649743.40659961
Sharpe:  0.8189274404511856
begin_total_asset:1000000
end_total_asset:4729661.643635573
Sharpe:  0.8241060665751208
begin_total_asset:1000000
end_total_asset:4729865.485147724
Sharpe:  0.8241285903783555
begin_total_asset:1000000
end_total_asset:4729874.907601955
Sharpe:  0.8241311937530685
----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 71        |
|    time_elapsed    | 162       |
|    total_timesteps | 11572     |
| train/             |           |
|    actor_loss      | -1.21e+08 |
|    critic_loss     | 1.06e+12  |
|    ent_coef        | 3.24      |
|    ent_coef_loss   | -185      |
|    learning_rate   | 0.0003    |
|    n_updates       | 11471     |
|    reward          | 4729875.0 |
----------------------------------
begin_total_asset:1000000
end_total_asset:4729447.187496673
Sharpe:  0.8240967803705357
begin_total_asset:1000000
end_total_

In [32]:
trained_sac.save('/content/trained_models/trained_sac.zip')

### Model 5: **TD3**


In [33]:
agent = DRLAgent(env = env_train)
TD3_PARAMS = {"batch_size": 100,
              "buffer_size": 1000000,
              "learning_rate": 0.001}

model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

{'batch_size': 100, 'buffer_size': 1000000, 'learning_rate': 0.001}
Using cuda device


In [34]:
trained_td3 = agent.train_model(model=model_td3,
                             tb_log_name='td3',
                             total_timesteps=30000)

begin_total_asset:1000000
end_total_asset:4305925.688121229
Sharpe:  0.7951663377013253
begin_total_asset:1000000
end_total_asset:4369803.080147342
Sharpe:  0.807342897272735
begin_total_asset:1000000
end_total_asset:4369803.080147342
Sharpe:  0.807342897272735
begin_total_asset:1000000
end_total_asset:4369803.080147342
Sharpe:  0.807342897272735
----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 120       |
|    time_elapsed    | 96        |
|    total_timesteps | 11572     |
| train/             |           |
|    actor_loss      | -5.81e+07 |
|    critic_loss     | 6.9e+11   |
|    learning_rate   | 0.001     |
|    n_updates       | 11471     |
|    reward          | 4369803.0 |
----------------------------------
begin_total_asset:1000000
end_total_asset:4369803.080147342
Sharpe:  0.807342897272735
begin_total_asset:1000000
end_total_asset:4369803.080147342
Sharpe:  0.807342897272735
begin_total_asset:1000

In [35]:
trained_td3.save('/content/trained_models/trained_td3.zip')

## Trading
Assume that we have $1,000,000 initial capital at 2019-01-01. We use the A2C model to trade Dow jones 30 stocks.

In [60]:
trade = data_split(df,'2020-07-01', '2021-10-31')
e_trade_gym = StockPortfolioEnv(df = trade, **env_kwargs)


In [61]:
trade.shape

(9436, 18)

In [62]:
df_daily_return, df_actions = DRLAgent.DRL_prediction(model=trained_a2c,
                        environment = e_trade_gym)

/usr/local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


begin_total_asset:1000000
end_total_asset:1371808.8229137172
Sharpe:  1.8388303368249521
hit end!


In [63]:
df_daily_return.head()

,date,daily_return
0,2020-07-01,0.000000
1,2020-07-02,0.004146
2,2020-07-06,0.014307
3,2020-07-07,-0.011319
4,2020-07-08,0.005393


from matplotlib import pyplot as plt
_df_0['daily_return'].plot(kind='hist', bins=20, title='daily_return')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('date').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['date']
  ys = series['daily_return']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_2.sort_values('date', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('date')):
  _plot_series(series, series_name, i)
  fig.legend(title='date', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('date')
_ = plt.ylabel('daily_return')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['date']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'date'}, axis=1)
              .sort_values('date', ascending=True))
  xs = counted['date']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_3.sort_values('date', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('date')):
  _plot_series(series, series_name, i)
  fig.legend(title='date', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('date')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_4['daily_return'].plot(kind='line', figsize=(8, 4), title='daily_return')
plt.gca().spines[['top', 'right']].set_visible(False)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_5['date'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_5, x='daily_return', y='date', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [64]:
df_daily_return.to_csv('df_daily_return.csv')

In [65]:
df_actions.head()

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,GS,...,MMM,MRK,MSFT,NKE,PG,TRV,UNH,VZ,WBA,WMT
date,,,,,,,,,,,,,,,,,,,,,
2020-07-01,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714,...,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714,0.035714
2020-07-02,0.028144,0.076505,0.028144,0.029835,0.047584,0.028144,0.028144,0.028144,0.053212,0.028144,...,0.028144,0.028176,0.051224,0.049945,0.028144,0.028144,0.063520,0.028682,0.028144,0.062643
2020-07-06,0.028144,0.076505,0.028144,0.029835,0.047584,0.028144,0.028144,0.028144,0.053212,0.028144,...,0.028144,0.028176,0.051224,0.049945,0.028144,0.028144,0.063520,0.028682,0.028144,0.062643
2020-07-07,0.028144,0.076505,0.028144,0.029835,0.047584,0.028144,0.028144,0.028144,0.053212,0.028144,...,0.028144,0.028176,0.051224,0.049945,0.028144,0.028144,0.063520,0.028682,0.028144,0.062643
2020-07-08,0.028144,0.076505,0.028144,0.029835,0.047584,0.028144,0.028144,0.028144,0.053212,0.028144,...,0.028144,0.028176,0.051224,0.049945,0.028144,0.028144,0.063520,0.028682,0.028144,0.062643


In [66]:
df_actions.to_csv('df_actions.csv')

<a id='6'></a>
# Part 7: Backtest Our Strategy
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

<a id='6.1'></a>
## 7.1 BackTestStats
pass in df_account_value, this information is stored in env class


In [67]:
from pyfolio import timeseries
DRL_strat = convert_daily_return_to_pyfolio_ts(df_daily_return)
perf_func = timeseries.perf_stats
perf_stats_all = perf_func( returns=DRL_strat,
                              factor_returns=DRL_strat,
                                positions=None, transactions=None, turnover_denom="AGB")

In [68]:
print("==============DRL Strategy Stats===========")
perf_stats_all

==============DRL Strategy Stats===========


,0
Annual return,0.266673
Cumulative returns,0.371809
Annual volatility,0.133447
Sharpe ratio,1.838830
Calmar ratio,3.168407
Stability,0.900235
Max drawdown,-0.084166
Omega ratio,1.357062
Sortino ratio,2.767603
Skew,-0.228574


In [69]:
#baseline stats
print("==============Get Baseline Stats===========")
baseline_df = get_baseline(
        ticker="^DJI",
        start = df_daily_return.loc[0,'date'],
        end = df_daily_return.loc[len(df_daily_return)-1,'date'])

stats = backtest_stats(baseline_df, value_col_name = 'close')

==============Get Baseline Stats===========


[*********************100%***********************]  1 of 1 completed

Shape of DataFrame:  (336, 8)
Annual return          0.279047
Cumulative returns     0.388402
Annual volatility      0.139129
Sharpe ratio           1.844560
Calmar ratio           3.124551
Stability              0.918675
Max drawdown          -0.089308
Omega ratio            1.358960
Sortino ratio          2.734872
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.052781
Daily value at risk   -0.016510
dtype: float64


<a id='6.2'></a>
## 7.2 BackTestPlot

In [70]:
import pyfolio
%matplotlib inline

baseline_df = get_baseline(
        ticker='^DJI', start=df_daily_return.loc[0,'date'], end='2021-11-01'
    )

baseline_returns = get_daily_return(baseline_df, value_col_name="close")

with pyfolio.plotting.plotting_context(font_scale=1.1):
        pyfolio.create_full_tear_sheet(returns = DRL_strat,
                                       benchmark_rets=baseline_returns, set_context=False)

[*********************100%***********************]  1 of 1 completed

Shape of DataFrame:  (337, 8)


AttributeError: 'Series' object has no attribute 'iteritems'

## Min-Variance Portfolio Allocation

In [71]:
%pip install PyPortfolioOpt

In [72]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models

In [73]:
unique_tic = trade.tic.unique()
unique_trade_date = trade.date.unique()

In [74]:
df.head()

,date,tic,adjcp,close,high,low,open,volume,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,cov_list,return_list
0,2008-12-31,AAPL,2.574141,3.048214,3.133571,3.047857,3.070357,607541200.0,-0.097446,3.649552,2.895305,42.254771,-80.847207,16.129793,3.243631,3.375887,"[[0.001348968986171653, 0.00042841264280825875...",tic AAPL AMGN AXP ...
1,2008-12-31,AMGN,40.937153,57.750000,58.220001,57.060001,57.110001,6287200.0,0.216368,58.947401,56.388599,51.060614,51.895357,10.432018,56.671334,56.044333,"[[0.001348968986171653, 0.00042841264280825875...",tic AAPL AMGN AXP ...
2,2008-12-31,AXP,14.445879,18.549999,18.750000,17.910000,17.969999,9625600.0,-1.191668,23.723023,16.106977,42.521170,-74.811722,25.776759,20.030000,22.412000,"[[0.001348968986171653, 0.00042841264280825875...",tic AAPL AMGN AXP ...
3,2008-12-31,BA,32.005898,42.669998,43.049999,41.500000,41.590000,5443100.0,-0.391219,42.894634,38.486366,47.290375,157.922391,5.366299,40.432000,43.304500,"[[0.001348968986171653, 0.00042841264280825875...",tic AAPL AMGN AXP ...
4,2008-12-31,CAT,29.245972,44.669998,45.099998,43.700001,43.700001,6277400.0,0.979845,45.785565,38.404435,51.073052,98.904653,26.331746,40.266000,39.918333,"[[0.001348968986171653, 0.00042841264280825875...",tic AAPL AMGN AXP ...


In [75]:
#calculate_portfolio_minimum_variance
portfolio = pd.DataFrame(index = range(1), columns = unique_trade_date)
initial_capital = 1000000
portfolio.loc[0,unique_trade_date[0]] = initial_capital

for i in range(len( unique_trade_date)-1):
    df_temp = df[df.date==unique_trade_date[i]].reset_index(drop=True)
    df_temp_next = df[df.date==unique_trade_date[i+1]].reset_index(drop=True)
    #Sigma = risk_models.sample_cov(df_temp.return_list[0])
    #calculate covariance matrix
    Sigma = df_temp.return_list[0].cov()
    #portfolio allocation
    ef_min_var = EfficientFrontier(None, Sigma,weight_bounds=(0, 0.1))
    #minimum variance
    raw_weights_min_var = ef_min_var.min_volatility()
    #get weights
    cleaned_weights_min_var = ef_min_var.clean_weights()

    #current capital
    cap = portfolio.iloc[0, i]
    #current cash invested for each stock
    current_cash = [element * cap for element in list(cleaned_weights_min_var.values())]
    # current held shares
    current_shares = list(np.array(current_cash)
                                      / np.array(df_temp.close))
    # next time period price
    next_price = np.array(df_temp_next.close)
    ##next_price * current share to calculate next total account value
    portfolio.iloc[0, i+1] = np.dot(current_shares, next_price)

portfolio=portfolio.T
portfolio.columns = ['account_value']

In [76]:
portfolio.head()

,account_value
2020-07-01,1000000
2020-07-02,1005234.89169
2020-07-06,1014933.771709
2020-07-07,1014238.663323
2020-07-08,1012674.036292


In [77]:
a2c_cumpod =(df_daily_return.daily_return+1).cumprod()-1

In [78]:
min_var_cumpod =(portfolio.account_value.pct_change()+1).cumprod()-1

<ipython-input-78-2a1c4bec35f1>:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  min_var_cumpod =(portfolio.account_value.pct_change()+1).cumprod()-1


In [79]:
dji_cumpod =(baseline_returns+1).cumprod()-1

## Plotly: DRL, Min-Variance, DJIA

In [80]:
%pip install plotly

  Using cached tenacity-9.0.0-py3-none-any.whl.metadata (1.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 62.5 MB/s eta 0:00:00
Using cached tenacity-9.0.0-py3-none-any.whl (28 kB)


In [81]:
from datetime import datetime as dt

import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go

In [82]:
time_ind = pd.Series(df_daily_return.date)

In [83]:
trace0_portfolio = go.Scatter(x = time_ind, y = a2c_cumpod, mode = 'lines', name = 'A2C (Portfolio Allocation)')

trace1_portfolio = go.Scatter(x = time_ind, y = dji_cumpod, mode = 'lines', name = 'DJIA')
trace2_portfolio = go.Scatter(x = time_ind, y = min_var_cumpod, mode = 'lines', name = 'Min-Variance')
#trace3_portfolio = go.Scatter(x = time_ind, y = ddpg_cumpod, mode = 'lines', name = 'DDPG')
#trace4_portfolio = go.Scatter(x = time_ind, y = addpg_cumpod, mode = 'lines', name = 'Adaptive-DDPG')
#trace5_portfolio = go.Scatter(x = time_ind, y = min_cumpod, mode = 'lines', name = 'Min-Variance')

#trace4 = go.Scatter(x = time_ind, y = addpg_cumpod, mode = 'lines', name = 'Adaptive-DDPG')

#trace2 = go.Scatter(x = time_ind, y = portfolio_cost_minv, mode = 'lines', name = 'Min-Variance')
#trace3 = go.Scatter(x = time_ind, y = spx_value, mode = 'lines', name = 'SPX')

In [84]:
fig = go.Figure()
fig.add_trace(trace0_portfolio)

fig.add_trace(trace1_portfolio)

fig.add_trace(trace2_portfolio)



fig.update_layout(
    legend=dict(
        x=0,
        y=1,
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=15,
            color="black"
        ),
        bgcolor="White",
        bordercolor="white",
        borderwidth=2

    ),
)
#fig.update_layout(legend_orientation="h")
fig.update_layout(title={
        #'text': "Cumulative Return using FinRL",
        'y':0.85,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
#with Transaction cost
#fig.update_layout(title =  'Quarterly Trade Date')
fig.update_layout(
#    margin=dict(l=20, r=20, t=20, b=20),

    paper_bgcolor='rgba(1,1,0,0)',
    plot_bgcolor='rgba(1, 1, 0, 0)',
    #xaxis_title="Date",
    yaxis_title="Cumulative Return",
xaxis={'type': 'date',
       'tick0': time_ind[0],
        'tickmode': 'linear',
       'dtick': 86400000.0 *80}

)
fig.update_xaxes(showline=True,linecolor='black',showgrid=True, gridwidth=1, gridcolor='LightSteelBlue',mirror=True)
fig.update_yaxes(showline=True,linecolor='black',showgrid=True, gridwidth=1, gridcolor='LightSteelBlue',mirror=True)
fig.update_yaxes(zeroline=True, zerolinewidth=1, zerolinecolor='LightSteelBlue')

fig.show()